In [ ]:
import time
import hopsworks
import os
from dotenv import load_dotenv
import joblib

# from datetime import datetime, timedelta
import pandas as pd

import hopsworks
import matplotlib.pyplot as plt

from utils import *
import utils


In [ ]:
model_version = 4
fng_feature_group_version = 1
solana_feature_group_version = 1
bitcoin_feature_group_version = 1

In [ ]:
today = int(time.time())
seconds_in_one_day = 24 * 60 * 60
today

In [ ]:
load_dotenv()
#os.environ["HOPSWORKS_API_KEY"] = os.getenv("HOPSWORKS_API_KEY")
project = hopsworks.login()
#secrets = secrets_api(project.name)
# This line will fail if you have not registered the CRYPTO_API_KEY as a secret in Hopsworks
#CRYPTO_API_KEY = secrets.get_secret("CRYPTO_API_KEY").value

#CRYPTO_API_KEY = os.getenv("CRYPTO_API_KEY")

In [ ]:
fs = project.get_feature_store() 
starting_timestamp = float(today - 40 * seconds_in_one_day)
starting_timestamp


In [ ]:
solana_feature_group = fs.get_feature_group(
    name="solana",  
    version=solana_feature_group_version
)

solana_batch_data = solana_feature_group.filter(solana_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
solana_batch_data.head()

In [ ]:
bitcoin_feature_group = fs.get_feature_group(
    name="bitcoin",  
    version=bitcoin_feature_group_version
)
bitcoin_batch_data = bitcoin_feature_group.filter(bitcoin_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
bitcoin_batch_data = bitcoin_batch_data[["timestamp","open"]]
bitcoin_batch_data.head()

In [ ]:
fng_feature_group = fs.get_feature_group(
    'f_n_g_index',
    version=fng_feature_group_version
)
fng_batch_data = fng_feature_group.filter(fng_feature_group.timestamp >= starting_timestamp).read().sort_values(by='timestamp', inplace=False).reset_index(drop=True)
fng_batch_data['fng_classification'] = fng_batch_data['fng_classification'].map(fng_labels_to_numbers_map)
fng_batch_data.head()

In [ ]:
from tensorflow.keras.models import load_model

mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="solana_price_daily_model_10d",
    version=model_version,
)
saved_model_dir = retrieved_model.download()
print(saved_model_dir)
model = load_model(saved_model_dir+'/best_model.keras')
model


In [ ]:
all_data = pd.merge(solana_batch_data, bitcoin_batch_data, how='inner', on="timestamp",suffixes=('_solana', '_bitcoin'))  
all_data = pd.merge(all_data, fng_batch_data, how='inner', on="timestamp")

if "timestamp" in all_data.columns:
    all_data = all_data.drop(columns=['timestamp'])

target_column = 'open_solana'
all_data = utils.move_target_at_the_end(all_data, target_column)
all_data_values = all_data.values[-30:]

all_data.head()

In [ ]:
model_workspace = "solana_price_10d_scalers/" 
scaler_labels = joblib.load(model_workspace+'scaler_labels.pkl')
scaler_features = joblib.load(model_workspace+'scaler_features.pkl')

all_data_values = scaler_features.transform(all_data_values)
X = all_data_values.reshape(1, 30, 10)


In [ ]:
all_data.columns

In [ ]:
predictions = model.predict(X)

In [ ]:
predictions_unscaled =  scaler_labels.inverse_transform(predictions)
predictions_unscaled


In [ ]:

timestamps = [today + (i+1) * seconds_in_one_day for i in range(len(predictions[0]))]

In [ ]:
monitor_fg = fs.get_or_create_feature_group(
    name='solana_predictions',
    description='Solana_predictions',
    version=1,
    primary_key=['timestamp']
)

In [ ]:
result = pd.DataFrame(columns=['timestamp', "open_solana"])
result["timestamp"]=timestamps
result["open_solana"]=predictions_unscaled[0]
result= result.sort_values(by=['date'])
result

In [ ]:
result['date'] = pd.to_datetime(result['timestamp'], unit='s')
result['date'] = result['date'].dt.normalize()
# result.drop(columns=["timestamp"], inplace=True)

result

In [ ]:
#this is plane and we may have duplicates
#monitor_fg.insert(result)

In [ ]:
#this insert prevents duplicates
monitor_fg.insert(
    result,
    # overwrite=False,  # Do not overwrite all data
    operation="upsert",  # Use upsert to handle duplicates based on the primary key
    #storage=None  # Write to both online and offline storage
)